### Demonstration of Quantum Key Distribution with the Ekert 91 Protocol

Algorithm -
1. First generate the a maximally entangled qubit pair |psi+> = 1/root(2) * (|01> - |10>)
2. Send one qubit to Alice and one qubit to Bob
3. Both Alice and Bob perform their measurement and make the measurement bases public.
4. According to the new information obtained, a sifted key is created, which can be used for secure communication

Modification to this Algorithm (because we get only 1 quantum computer) -
1. First generate the a maximally entangled qubit pair |psi+> = 1/root(2) * (|01> - |10>)
2. Take the measurement bases from Alice and Bob
3. Perform measurement, and send the measurement results to Alice and Bob respectively
4. Note that Alice and Bob do not have each other's measurement outcomes, they have only theirs
5. The measurement bases are made public, and the sifted key is obtained

In [13]:
import os
from qiskit import execute
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit

from quantuminspire.credentials import get_authentication, save_account
from quantuminspire.qiskit import QI
from apikey import token

QI_URL = os.getenv('API_URL', 'https://api.quantum-inspire.com/')

import re
import numpy as np
import random
print("Process Complete!")

Process Complete!


<a id="layout"></a>
# 1. Quantum Key Distribution Activity layout

In project, we are going to implement the E91 protocol. Steps of the protocol:

1. The serves creates a singlet state where one pair corresponds to Alice and other to Bob.
2. Alice randomly selects a sequence of measurement basis from Z,X,V and sends it to server
3. Bob randomly selects a sequence of measurement basis from W,V,X and sends it to server
4. `Intermediate interface function 1`: a MUX that creates a circuit based on Alice and Bob's selection of bases
5. `Intermediate interface function 2`: execute the measurement on Quantum Inspire 
6. `Intermediate interface function 3`: check the basis and create the key and send it to Alice and Bob 

These 6 steps allow a key to be distributed between Alice and Bob securely, now the two can send secure and encrypted messages through an insecure channel. 

In this lab, we will not worry about an eavesdropper, but focus on the code for the basic protocol. Therefore, Alice and Bob don't need to run an analysis step. We can further extend it this to try implementing code for Eve.

In [14]:
# Parameters
N_en_pairs = 10
alice_seq = [random.randint(1, 3) for i in range(N_en_pairs)]
bob_seq = [random.randint(1, 3) for i in range(N_en_pairs)]
print("Process Complete!")

Process Complete!


<a id="layout"></a>
## 1. Create entangled states and encode measurement sequence

In [15]:
Quantum_Circuit = [] # list for storing the quantum circuit for each bit

for i in range(N_en_pairs):
    Alice_Reg = QuantumRegister(1, name="alice")
    Bob_Reg = QuantumRegister(1, name="bob")
    cr = ClassicalRegister(2, name="cr")
    qc = QuantumCircuit(Alice_Reg, Bob_Reg, cr)
    
     # Create an entangled pair for Alice and Bob in each loop
    qc.x(Alice_Reg)
    qc.x(Bob_Reg)
    qc.h(Alice_Reg)
    qc.cx(Alice_Reg, Bob_Reg)
    
    # Cicuit Measurement for different bases
    
    if alice_seq[i]== 1:            #If Alice's random sequence is 1, Alice measures in the Z basis
        qc.measure(Alice_Reg,cr[0])  
    elif alice_seq[i] == 2:         #If Alice's random sequence is 2, Alice measures in the X basis
        qc.h(Alice_Reg)
        qc.measure(Alice_Reg,cr[0])
    elif alice_seq==3:              #If Alice's random sequence is 3, Alice measures in the V basis (-1/sqrt(2), 0, 1/sqrt(2))
        qc.s(Alice_Reg)
        qc.h(Alice_Reg)
        qc.tdg(Alice_Reg)
        qc.h(Alice_Reg)
        qc.measure(Alice_Reg, cr[0])
        
    if bob_seq[i]==1:               #If  Bob's random sequence is 1, Bob measures in the -W basis
        qc.s(Bob_Reg)
        qc.h(Bob_Reg)
        qc.t(Bob_Reg)
        qc.h(Bob_Reg)
        qc.measure(Bob_Reg, cr[1])
    elif bob_seq[i] == 2:           #If  Bob's random sequence is 2, Bob measures in the V basis
        qc.s(Bob_Reg)
        qc.h(Bob_Reg)
        qc.tdg(Bob_Reg)
        qc.h(Bob_Reg)
        qc.measure(Bob_Reg, cr[1])
    elif bob_seq[i] == 3:           #If  Bob's random sequence is 3, Bob measures in the X basis
        qc.h(Bob_Reg)
        qc.measure(Bob_Reg, cr[1])

    Quantum_Circuit.append(qc)

print("Process Complete!")

Process Complete!


In [16]:
Quantum_Circuit[0].draw(output='mpl')
print("Process Complete!")

Process Complete!


## To QI

In [17]:
save_account(token)
project_name = 'E91_test_hardware'
authentication = get_authentication()
QI.set_authentication()

# Create an interface between Qiskit and Quantum Inpsire to execute the circuit
# qi_backend = QI.get_backend('Starmon-5')
qi_backend = QI.get_backend('QX single-node simulator')

job = execute(Quantum_Circuit, qi_backend, shots = 1)
print("Process Complete!")

Process Complete!


In [18]:
results = job.result()
counts = results.get_counts()
print("Process Complete!")

Process Complete!


In [19]:
abPatterns = [
    re.compile('00'), # search for the '..00' output (Alice obtained -1 and Bob obtained -1)
    re.compile('01'), # search for the '..01' output
    re.compile('10'), # search for the '..10' output (Alice obtained -1 and Bob obtained 1)
    re.compile('11')  # search for the '..11' output
]
print("Process Complete!")

Process Complete!


### Alices and Bobs measurement result

In [20]:
aliceResults = [] # Alice's results (string a)
bobResults = [] # Bob's results (string a')

for i in range(N_en_pairs):

    res = list(counts[i].keys())[0] # extract the key from the dict and transform it to str; execution result of the i-th circuit
    
    if abPatterns[0].search(res): # check if the key is '..00' (if the measurement results are -1,-1)
        aliceResults.append(-1) # Alice got the result -1 
        bobResults.append(-1) # Bob got the result -1
    if abPatterns[1].search(res):
        aliceResults.append(1)
        bobResults.append(-1)
    if abPatterns[2].search(res): # check if the key is '..10' (if the measurement results are -1,1)
        aliceResults.append(-1) # Alice got the result -1 
        bobResults.append(1) # Bob got the result 1
    if abPatterns[3].search(res): 
        aliceResults.append(1)
        bobResults.append(1)

print("Process Complete!")

Process Complete!


### Key Generation

In [21]:
aliceKey = [] # Alice's key string k
bobKey = [] # Bob's key string k'

# comparing the strings with measurement choices
for i in range(N_en_pairs):
    # if Alice and Bob have measured the spin projections onto the a_2/b_3 or a_3/b_2 directions
    if (alice_seq[i] == 2 and bob_seq[i] == 3) or (alice_seq[i] == 3 and bob_seq[i] == 2):
        aliceKey.append(aliceResults[i]) # record the i-th result obtained by Alice as the bit of the secret key k
        bobKey.append(- bobResults[i]) # record the multiplied by -1 i-th result obtained Bob as the bit of the secret key k'
        
keyLength = len(aliceKey) # length of the secret key

print(aliceKey)
print("Process Complete!")

[-1, -1]
Process Complete!


In [22]:
abKeyMismatches = 0 # number of mismatching bits in Alice's and Bob's keys

for j in range(keyLength):
    if aliceKey[j] != bobKey[j]:
        abKeyMismatches += 1

print(abKeyMismatches)
print("Process Complete!")

0
Process Complete!


Thus, Quantum Key Distribution has been demonstrated!